## Import Libraries

In [1]:
import matplotlib.pyplot as plt
from numpy import sin, cos, sqrt, pi, linspace, arange, deg2rad, rad2deg, array, arcsin, arccos,sort, argsort, argwhere, argmin, argmax, interp, concatenate, zeros
from scipy.spatial import distance
import warnings
from numpy import linalg as LA
warnings.simplefilter('ignore')

def sec(x):
    return 1/cos(x)
def tan(x):
    return sin(x)/cos(x)

## Initialize Parameters 

In [2]:
n      = 1.33 #-------------------------------------------- Refractive index of medium
c      = 299792458/n #------------------------------------- Speed of light in medium
R      = 7.3/2 #------------------------------------------- Radius of tank
v      = n * c #------------------------------------------- Particle Speed
times  = linspace(7e-11,1e-7,200000)

c1       = (0, 0, 0)  #------------------------------------ Central    PMT number 1 
c2       = (1.85*cos(2*pi/3) , 1.85*sin(2*pi/3), 0) #------ Non-Radial PMT number 2
c3       = (1.85*cos(4*pi/3) , 1.85*sin(4*pi/3), 0) #------ Non-Radial PMT number 3
c4       = (1.85*cos(0     ) , 1.85*sin(0     ), 0) #------ Radial     PMT number 4
xA,xB,h  = R-3.0, -(R-6.5), 1.3 #-------------------------- PARAM

A   = array( [xA,  sqrt(R**2-xA**2), h] ) #---------------- Entry Point of muon
B   = array( [xB,  -sqrt(R**2-xB**2), h] ) #---------------- Exit point of muon
AB  = B-A #------------------------------------------------ Displacement vector of muon
nAB = LA.norm(AB) #---------------------------------------- Path length traveled by muon
print(nAB)
den = c*c - v*v
print('A : ', A)
print('B : ', B)

6.270604132263064
A :  [0.65     3.591657 1.3     ]
B :  [ 2.85       -2.28035085  1.3       ]


In [3]:
def entry_brightness(L,c,v,alpha,den):
    x  = 0
    tt = L/c
    bigterm = sqrt( v*v*( -L*L*v*v + c*c*L*L + c*c*tt*tt*v*v - 2*c*c*L*tt*v*cos(alpha) + L*L*v*v*cos(alpha)**2) )
    xp      = ( c*c*tt*v-L*v*v*cos(alpha) + bigterm ) / den
    vp      = ( ( c*c*v) + ( (c*c*v**3*(tt*v-L*cos(alpha)))  / bigterm ) ) / den
    kp      = sqrt( L*L + xp*xp - 2*L*xp*cos(alpha) )
    betap   = pi - (  arccos( (xp*xp + kp*kp - L*L) / (2*xp*kp) )  )
    vtp     = vp*sin(betap)
    omegap  = vtp / kp
    bp      = abs( omegap / (kp**2) )
    return bp

global nls
nls='--'

def plus_b_vs_t(tp,bp,color,mylabel,xc):
    plt.plot(tp, bp, c=color, ls='-', label=mylabel, lw=2.5)
    plt.xlabel(r'time (ns)', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '        , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
    #plt.xlim(-0.1,6.3)
def minus_b_vs_t(tm,bm,color,mylabel,xc):
    plt.plot(tm, bm, c=color, ls=nls, label=mylabel, lw=2.5)
    plt.xlabel(r'time (ns)', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '                   , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
    #plt.xlim(-0.1,6.3)
def both_b_vs_t(tp,bp,tm,bm,color,mylabel,xc):
    plt.plot(tp, bp, c=color, ls='-', label=mylabel, lw=2.5)
    plt.plot(tm, bm, c=color, ls=nls               , lw=2.5)
    plt.xlabel(r'time (ns)', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '                   , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
    #plt.xlim(-0.1,6.3)

In [5]:
%matplotlib
plt.figure()

plotme    = 'b vs t'

detector_coordinates = [c1,c2,c3,c4]
colors               = ['k','r','b','g']
detectors            = [1,2,3,4]
labels               = ['Central Detector', "Detector 2",'Detector 3','Detector 4']

for D, color, detector, mylabel in zip(detector_coordinates, colors, detectors, labels):
    
    print("==============================================")
    print("Processing detector %s"%detector)
    
    AD    = D - A
    BD    = D - B
    L     = LA.norm(AD)
    alpha = arccos((sum(AD*AB))/(L*nAB)) # OR use alpha1 = arccos( (L*L+x*x-k*k)/(2*x*L) )
    
    xc    = L*cos(alpha) - (c*L*sin(alpha))/sqrt(-den)    
    print('Critical distance (from entry point) : {} m'.format(xc))

    T,XP,XM,BP,BM,BETAP,BETAM=[],[],[],[],[],[],[]
    
    for iii,t in enumerate(times):

        x      = v*t
        ratio  = x/nAB
        X      = array([ (1-ratio)*A[0] + ratio*B[0] , (1-ratio)*A[1] + ratio*B[1] , h   ])
        AX,DX  = X - A, X-D
        k      = LA.norm(DX)
        t1,t2  = t, k/c
        tt     = t1 + t2
        xp     = (c*c*tt*v-L*v*v*cos(alpha)+sqrt(v*v*( -L*L*v*v + c*c*L*L + c*c*tt*tt*v*v - 2*c*c*L*tt*v*cos(alpha) + L*L*v*v*cos(alpha)**2))) / den
        xm     = (c*c*tt*v-L*v*v*cos(alpha)-sqrt(v*v*( -L*L*v*v + c*c*L*L + c*c*tt*tt*v*v - 2*c*c*L*tt*v*cos(alpha) + L*L*v*v*cos(alpha)**2))) / den
        vp     = ( (c*c*v) + ( (c*c*v**3*(tt*v-L*cos(alpha)))  / (sqrt( v*v*(-L*L*v*v + c*c*(L*L+tt*tt*v*v) - 2*c*c*L*tt*v*cos(alpha) + L*L*v*v*cos(alpha)**2) )) )  ) / den
        vm     = ((-c*c*v) + ( (c*c*v**3*(tt*v-L*cos(alpha)))  / (sqrt( v*v*(-L*L*v*v + c*c*(L*L+tt*tt*v*v) - 2*c*c*L*tt*v*cos(alpha) + L*L*v*v*cos(alpha)**2) )) )  ) / den 
        kp     = sqrt( L*L + xp*xp - 2*L*xp*cos(alpha) )
        km     = sqrt( L*L + xm *xm  - 2*L *xm*cos(alpha) )
        betap  = pi - arccos( (xp*xp + kp*kp - L*L) / (2*xp*kp) )  
        betam  = pi - arccos( (xm *xm  + km *km  - L*L) / (2*xm *km ) )  
        vtp    = vp*sin(betap)
        vtm    = vm *sin(betam )
        omegap = vtp / kp
        omegam = vtm  / km
        bp,bm  = abs(omegap/(kp**2)), abs(omegam/(km **2))
        if ((iii+1)%20000==0):
            print(x,xp,xm,sep='\t')
        XP.append(xp); XM.append(xm); BP.append(bp); BM.append(bm); BETAP.append(rad2deg(betap)); BETAM.append(rad2deg(betam)); T.append(tt)

    XP, XM, BP, BM, T, BETAP, BETAM = array(XP), array(XM), array(BP), array(BM), array(T), array(BETAP), array(BETAM)
    Tn  = T * 1e+9 ; TT  = Tn #(Tn-Tn.min())

    cond  = ( ((XP>=0) & (XP<=nAB)) | ((XM>=0) & (XM<=nAB)))
    TT    = TT[cond] ; BP = BP[cond] ; BM = BM[cond] ; XP = XP[cond]
    XM    = XM[cond] ; BETAP=BETAP[cond] ; BETAM=BETAM[cond]
    
    conp = [(XP>=0) & (XP<=nAB)]
    conm  = [(XM >=0) & (XM <=nAB)]
    pluslen = len (XP[conp])
    minuslen  = len (XM [conm ])
    
    if detector==1:
        Bnorm = 1#entry_brightness(L,c,v,alpha,den)
        
    if pluslen == 0:
        if minuslen == 0:
            print("Images outside tank. Skipping...")
        else:
            print('Only image moving towards exit. Processing...')
            TT,XM,BM,BETAM = TT[conm], XM[conm], BM[conm]/Bnorm, BETAM[conm]
            if plotme=='b vs x':
                minus_b_vs_x(xm=XM,bm=BM,color=color,mylabel=mylabel,xc=xc)
            elif plotme=='b vs t':
                minus_b_vs_t(tm=TT,bm=BM,color=color,mylabel=mylabel,xc=xc)
    elif pluslen != 0:
        if minuslen == 0:
            print('Only image moving towards entry. Processing...')
            TT,XP,BP,BETAP = TT[conp], XP[conp], BP[conp]/Bnorm, BETAP[conp]
            if plotme=='b vs x':
                plus_b_vs_x(xp=XP,bp=BP,color=color,mylabel=mylabel,xc=xc)
            elif plotme=='b vs t':
                plus_b_vs_t(tp=TT,bp=BP,color=color,mylabel=mylabel,xc=xc)

        else:
            print('Both images moving. Processing...')
            TTm, XM, BM, BETAM = TT[conm], XM[conm], BM[conm]/Bnorm, BETAM[conm]
            TTp, XP, BP, BETAP = TT[conp], XP[conp], BP[conp]/Bnorm, BETAP[conp]
            if plotme=='b vs x':
                both_b_vs_x(xp=XP,bp=BP,xm=XM,bm=BM,color=color,mylabel=mylabel,xc=xc)
            elif plotme=='b vs t':
                if detector==4:
                    both_b_vs_t(tp=TTp,bp=BP,tm=TTm,bm=BM,color='g',mylabel=mylabel,xc=xc)
                else:
                    both_b_vs_t(tp=TTp,bp=BP,tm=TTm,bm=BM,color=color,mylabel=mylabel,xc=xc)
plt.axhline(1,c='k',ls=':')
plt.axvline(0,c='k',ls=':')
#plt.xlim(-0.05,6.3)
plt.legend(loc='upper right', prop={'size': 14})
plt.show()

Using matplotlib backend: Qt5Agg
Processing detector 1
Critical distance (from entry point) : 0.5391523328682735 m
3.01667669200846	-4.3236682894734315	3.01667669200846
6.012517704007521	-19.918370288725967	6.012517704007521
9.00835871600658	-39.8049718586298	9.008358716006578
12.00419972800564	-60.47911370134385	12.004199728005633
15.0000407400047	-81.38564743442359	15.000040740004696
17.995881752003758	-102.3891184011513	17.995881752003772
20.991722764002816	-123.44184171244386	20.99172276400283
23.98756377600188	-144.5229205987139	23.987563776001874
26.98340478800094	-165.62179111723273	26.983404788000925
29.979245799999998	-186.7325486949348	29.979245799999987
Both images moving. Processing...


NameError: name 'both_b_vs_t' is not defined

In [5]:
plt.plot(TTp,BP)
plt.plot(TTm,BM)
plt.yscale('log')

In [6]:
TT

array([ 1.297725  ,  1.2975969 ,  1.29746881, ..., 16.92819226,
       16.92928009, 16.93036794])